In [8]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import os
import pandas as pd
import re
from datetime import datetime
import time

import os
from io import BytesIO
from openpyxl.drawing.image import Image
import PIL.Image
from PIL import Image
from fpdf import FPDF

from upload_data import *

#-------web driver setting-----------
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

chrome_driver_path = r'C:/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

In [4]:
meta = pd.read_parquet("data.parquet")
meta

,product_names,product_urls,product_prices,prodict_sellers,product_detail,product_marketings
0,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,https://cr.shopping.naver.com/adcr.nhn?x=dVYa9...,"2,000원",스카이트립,"종류 : 콘센트형, 전원스위치 : 통합스위치, 코드수 : 1구",
1,감비앤코 여행용 멀티 어댑터 해외여행 멀티탭 돼지코 전세계 멀충 PD C타입,https://cr.shopping.naver.com/adcr.nhn?x=cM9sa...,"32,000원",감비앤코,"종류 : 콘센트형, 전원스위치 : 스위치없음, 코드수 : 1구, 특징 : 과부하차단...",출시기념 특가 행사와 파우치 증정 이벤트
2,지티스 여행용 멀티 어댑터 해외여행 C타입 초고속 전세계 110V 멀티탭,https://cr.shopping.naver.com/adcr.nhn?x=Kve9j...,"31,800원",지티스몰,"종류 : 콘센트형, 전원스위치 : 스위치없음, 코드수 : 1구, 특징 : 과부하차단...",알림받기 즉시할인 및 포토영상 리뷰 작성 시 즉시! 1000포인트 지급
3,여행용 멀티 어댑터 플러그 해외여행 트래블어댑터 콘센트,https://cr.shopping.naver.com/adcr.nhn?x=xhcVC...,"10,900원",파워존 스토어,"종류 : 콘센트형, 전원스위치 : 스위치없음, 특징 : 과부하차단",소식알림 할인쿠폰! 포토리뷰 300포인트 적립!
4,멀티어댑터 여행용 어댑터 GaN PPS USB 5포트 GAN PD 65W 파우치 기본구성,https://cr.shopping.naver.com/adcr.nhn?x=z8fZ7...,"39,900원",Dibea,"종류 : 콘센트형, 코드수 : 1구, 특징 : USB충전포트",
...,...,...,...,...,...,...
115,크리스탈클라우드 5포트 PD 70W GaN 전세계 여행용 멀티어댑터 CC-308,https://cr.shopping.naver.com/adcr.nhn?x=zSEBv...,"59,900원",오늘의집,,첫구매 시 최대 2만원 할인 / 네이버페이 최대 3000P 적립 / 토스페이 최대 ...
116,일본 미국 캐나다 대만 돼지코 변환 플러그 해외용 여행용 어댑터 110V 220V,https://cr.shopping.naver.com/adcr.nhn?x=iCIET...,"3,900원",까만몰,,
117,모락 브리큐 USB 4포트 여행용 멀티 플러그 어댑터 MR-BRIQ01,https://cr.shopping.naver.com/adcr.nhn?x=%2BTc...,"24,900원",morac,"종류 : 콘센트형, 전원스위치 : 스위치없음, 코드수 : 4구, 특징 : 과부하차단...",
118,가우넷 오호 YM105 USB 4포트 C타입 여행용 어댑터 플러그,https://cr.shopping.naver.com/adcr.nhn?x=ocT1b...,"23,900원",가우넷,"코드수 : 4구, 특징 : USB충전포트",


In [5]:
meta.drop_duplicates(subset= ['prodict_sellers'], inplace=True)

In [6]:
product_urls = meta['product_urls'].to_list()
product_urls = product_urls[60:]

In [1]:
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = product_urls

folder_paths = []

exception_urls = []

# URL에서 데이터 수집
for url in urls:
    driver.get(url)
    actual_url = driver.current_url
    print(actual_url)

    if not (actual_url.startswith('https://brand') or actual_url.startswith('https://smartstore')):
        exception_urls.append(actual_url)
        continue

    # 브랜드명 및 제품명 수집
    try:
        try:
            brand_element = driver.find_element(By.CSS_SELECTOR, '#pc-storeNameWidget > div > div > a > img')
            brand = brand_element.get_attribute('alt')
        except Exception as e1:
            try:
                brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > img")
                brand = brand_element.get_attribute('alt')
            except Exception as e2:
                try:
                    brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-storeNameWidget > div > div > a > span")
                    brand = brand_element.text
                except Exception as e3:
                    try:
                        brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > span")
                        brand = brand_element.text
                    except Exception as e4:
                        print(f"브랜드명 수집 실패 : {e4}")
                        exception_urls.append(url)
                        continue

        product = driver.find_element(By.CSS_SELECTOR, '#content > div > div._2-I30XS1lA > div._2QCa6wHHPy > fieldset > div._3k440DUKzy > div._1eddO7u4UC > h3').text
    except Exception as e:
        print(f"브랜드명 및 상품명 수집 실패 : {e}")
        exception_urls.append(url)
        continue

    sanitized_product = product.replace("/","_").replace(" ","_")
    folder_name = sanitized_product
    # folder_name = f"{brand}_{product}"
    folder_path = os.path.join("C:/Users/shopperhouse/Desktop/C_Image", folder_name) # local_path
    os.makedirs(folder_path, exist_ok=True)
    folder_paths.append((folder_name, folder_path))

    # 상세페이지 더보기 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")


    count = 0

    try:
        # 상세정보 버튼
        driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
        time.sleep(3)

        # 상세정보 펼쳐보기 버튼
        driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
        time.sleep(3)

    except Exception as e:
        print(f"상세정보 버튼 클릭 실패: {e}")
        exception_urls.append(url)
        continue


    # 셀레니움으로 html 가져오기
    html_source = driver.page_source

    # bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 상세정보 전체
    images = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'})

    # 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for image in range(len(images)):
        # 4-4. 이미지 수집
        image_tag = images[image].findAll('img', {'class': 'se-image-resource'})
        if image_tag:
            image_link = image_tag[0]['data-src']
            print(f'Review Image Link: {image_link}')  # 이미지 링크 출력
            image_link = image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            local_img_path = os.path.join(folder_path, f'{count}.jpg')
            print(local_img_path)
            urllib.request.urlretrieve(image_link, local_img_path)
            count += 1

            # # s3에도 업로드
            # ecommerce = 'naver'
            # crawling_type = 'image'
            # category = '전기용품'         # 여기부터 변경
            # search_type = 'keyword'
            # file_type = 'jpg'
            # jpg_fullname = f'{folder_name}/{count}'
            # jpg_file_path = local_img_path

            # save_image_jpg_data(ecommerce, crawling_type, category, search_type, file_type, jpg_file_path, jpg_fullname)

print('done')
driver.quit()

for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

     # S3에 업로드
    ecommerce = 'naver'
    crawling_type = 'image'
    category = '전기용품'
    file_type = 'pdf'
    ctg_fullname = folder_name
    search_type = 'keyword'
    save_image_pdf_data(ecommerce, crawling_type, category, file_type, pdf_output_path, ctg_fullname, search_type)

### PDF 변환 및 업로드 따로 실행

In [2]:
import os
base_folder_path = 'C:/C_Image'

# 하위 폴더들 탐색
folder_paths = [(folder_name, os.path.join(base_folder_path, folder_name)) 
                for folder_name in os.listdir(base_folder_path) 
                if os.path.isdir(os.path.join(base_folder_path, folder_name))]

In [3]:
for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

     # S3에 업로드
    ecommerce = 'naver'
    crawling_type = 'image'
    category = '전기용품'
    file_type = 'pdf'
    ctg_fullname = folder_name
    search_type = 'keyword'
    save_image_pdf_data(ecommerce, crawling_type, category, file_type, pdf_output_path, ctg_fullname, search_type)

In [4]:
for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")